In [5]:
pip install pymysql

  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/e5/30/20467e39523d0cfc2b6227902d3687a16364307260c75e6a1cb4422b0c62/PyMySQL-1.1.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd

import getpass  # to get the password without showing the input
password = getpass.getpass()



········


In [17]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [19]:
df_may = pd.read_sql_query('select * from rental_may', engine)
df_may.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,16050,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2023-12-19 16:53:14
1,16051,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2023-12-19 16:53:14
2,16052,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2023-12-19 16:53:14
3,16053,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2023-12-19 16:53:14
4,16054,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2023-12-19 16:53:14


In [20]:
df_june = pd.read_sql_query('select * from rental_june', engine)
df_june.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,16050,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2023-12-19 16:58:08
1,16051,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2023-12-19 16:58:08
2,16052,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2023-12-19 16:58:08
3,16053,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2023-12-19 16:58:08
4,16054,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2023-12-19 16:58:08


In [21]:
#Check the number of rentals for each customer for May.

pd.read_sql_query("select  count(rm.rental_id) as rental_count, rm.customer_id, c.first_name, c.last_name from rental_may as rm left join customer as c on rm.customer_id = c.customer_id group by  customer_id", engine)


,rental_count,customer_id,first_name,last_name
0,2,1,MARY,SMITH
1,1,2,PATRICIA,JOHNSON
2,2,3,LINDA,WILLIAMS
3,3,5,ELIZABETH,BROWN
4,3,6,JENNIFER,DAVIS
...,...,...,...,...
515,4,594,EDUARDO,HIATT
516,1,595,TERRENCE,GUNDERSON
517,6,596,ENRIQUE,FORSYTHE
518,2,597,FREDDIE,DUGGAN


In [23]:
pdread_sql_query("select  count(rj.rental_id) as rental_count, rj.customer_id, c.first_name, c.last_name from rental_june as rj left join customer as c on rj.customer_id = c.customer_id group by customer_id", engine)


,rental_count,customer_id,first_name,last_name
0,28,1,MARY,SMITH
1,4,2,PATRICIA,JOHNSON
2,16,3,LINDA,WILLIAMS
3,24,4,BARBARA,JONES
4,20,5,ELIZABETH,BROWN
...,...,...,...,...
585,8,595,TERRENCE,GUNDERSON
586,8,596,ENRIQUE,FORSYTHE
587,12,597,FREDDIE,DUGGAN
588,4,598,WADE,DELVALLE


In [43]:
#Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
merged_df = pd.merge(df_may, df_june, on='customer_id', how='outer', suffixes=('_may', '_june'))

In [45]:
merged_df['film_borrowed_in_may'] = merged_df.groupby('customer_id')['rental_id_may'].transform('count') 
merged_df['film_borrowed_in_june'] = merged_df.groupby('customer_id')['rental_id_june'].transform('count') 


In [46]:
comparison_df = merged_df[['customer_id', 'film_borrowed_in_may', 'film_borrowed_in_june']].drop_duplicates()
comparison_df 

,customer_id,film_borrowed_in_may,film_borrowed_in_june
0,130,48,48
48,459,28,28
76,408,36,36
112,333,16,16
128,222,40,40
...,...,...,...
18774,598,0,4
18778,487,0,4
18782,555,0,4
18786,335,0,4
